In [1]:
from topologicpy.Graph import Graph
from topologicpy.Helper import Helper
import topologicpy
from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.Dictionary import Dictionary
from topologicpy.Cluster import Cluster
from src.topologicpy.Edge import Edge

print(Helper.Version())
renderer = "browser"
print(topologicpy.__file__)  # Zeigt den Pfad der geladenen TopologicPy-Version


The version that you are using (0.8.15) is the latest version available on PyPI.
/Users/roger/miniconda3/lib/python3.12/site-packages/topologicpy/__init__.py


In [2]:
import topologic_core as topologic


def rectangle_face(min_x, min_y, max_x, max_y):
    """
    Manually build a rectangle face in the XY plane from (min_x, min_y) to (max_x, max_y).
    1. Make four vertices
    2. Make four edges
    3. Combine into a wire
    4. Make a face by Face.ByExternalBoundary
    """
    # 1. Corner vertices in 2D (z=0)
    vA = topologic.Vertex.ByCoordinates(min_x, min_y, 0)
    vB = topologic.Vertex.ByCoordinates(max_x, min_y, 0)
    vC = topologic.Vertex.ByCoordinates(max_x, max_y, 0)
    vD = topologic.Vertex.ByCoordinates(min_x, max_y, 0)

    # 2. Four edges
    eAB = topologic.Edge.ByStartVertexEndVertex(vA, vB)
    eBC = topologic.Edge.ByStartVertexEndVertex(vB, vC)
    eCD = topologic.Edge.ByStartVertexEndVertex(vC, vD)
    eDA = topologic.Edge.ByStartVertexEndVertex(vD, vA)

    # 3. Single wire from these edges
    rect_wire = topologic.Wire.ByEdges([eAB, eBC, eCD, eDA], False)

    # 4. Face from the wire
    rect_face = topologic.Face.ByExternalBoundary(rect_wire, False)
    return rect_face


def create_faces_and_shell():
    """
    Creates a single-floor 2 x 3 grid of 3×3 square faces (XY-plane),
    separated by 0.1 m, plus an enclosing bounding ring 0.3 m offset.
    
    Returns dict with keys:
      - "faces": a *LIST* of the 6 square faces
      - "boundingShell": a Face ring offset by 0.3 m
      - "allGeometry": everything as a single Cluster
    """

    cell_size = 3.0  # 3 m squares
    gap = 0.1  # 0.1 m gap in x,y
    num_x = 2
    num_y = 3
    offset_shell = 0.3  # extra boundary offset

    # 1. Build 6 squares in a 2×3 layout
    face_list = []
    for ix in range(num_x):
        for iy in range(num_y):
            # Compute lower-left corner of this square
            x0 = ix * (cell_size + gap)
            y0 = iy * (cell_size + gap)
            # Make a 3×3 face
            f = rectangle_face(x0, y0, x0 + cell_size, y0 + cell_size)

            # Attach a dictionary if you want
            d = topologic.Dictionary.ByKeysValues(
                ["ix", "iy"],
                [topologic.StringAttribute(str(ix)), topologic.StringAttribute(str(iy))]
            )
            f.SetDictionary(d)

            face_list.append(f)

    # 2. Gather all face vertices to find bounding box
    #    (We won't combine faces into a single cluster here, so we do it manually)
    all_verts = []
    for face in face_list:
        these_verts = []
        face.Vertices(None, these_verts)
        all_verts.extend(these_verts)

    xs = [v.X() for v in all_verts]
    ys = [v.Y() for v in all_verts]

    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)

    # 3. Create 'inner' rectangle exactly bounding the squares
    inner_rect = rectangle_face(min_x, min_y, max_x, max_y)

    # 4. Create 'outer' rectangle = expanded by 0.3 m
    out_min_x = min_x - offset_shell
    out_min_y = min_y - offset_shell
    out_max_x = max_x + offset_shell
    out_max_y = max_y + offset_shell
    outer_rect = rectangle_face(out_min_x, out_min_y, out_max_x, out_max_y)

    # 5. Difference => ring bounding shell
    bounding_shell = topologic.Topology.Difference(outer_rect, inner_rect)

    # 6. Combine everything into one cluster for "allGeometry"
    #    This cluster has the bounding shell face plus the 6 separate faces
    all_geo = topologic.Cluster.ByTopologies(face_list + [bounding_shell], False)

    return {
        "faces": face_list,  # <-- a plain Python list of faces
        "boundingShell": bounding_shell,  # the ring offset by 0.3
        "allGeometry": all_geo  # cluster of everything
    }

In [3]:
result = create_faces_and_shell()
faces = result["faces"]
shell = result["boundingShell"]
geo = result["allGeometry"]

In [4]:
Topology.Show(shell, faces, renderer=renderer)

In [5]:
print("Shell: " + shell.__str__())
print("Faces: " + faces.__str__())

Shell: <topologic_core.Face object at 0x13534ee70>
Faces: [<topologic_core.Face object at 0x13534c370>, <topologic_core.Face object at 0x13534c0b0>, <topologic_core.Face object at 0x13534c270>, <topologic_core.Face object at 0x13534c230>, <topologic_core.Face object at 0x13534f1b0>, <topologic_core.Face object at 0x13534c130>]


In [6]:
test_face = faces[0]
edges = Face.Edges(test_face)
side_length = Edge.Length(edges[0])
print("Side length of one interior square:", side_length)

Side length of one interior square: 3.0
